### Importing Main Libraries:

In [3]:
%matplotlib notebook
import IPython
from IPython.display import display
from sqlalchemy import create_engine
import psycopg2
import psycopg2.extras
import pandas as pd
import csv
from numpy import nan as NA
from datetime import datetime
import re
import sys
import numpy as np
import matplotlib.pyplot as plt 
from pandas import *
import pickle
import requests
import os

# ------------------------DATA INGESTION------------------------

First of all, we will change pandas settings so it can show all the features.

In [4]:
#Panda settings
#Pandas will not display all columns in our data when using the head() function without this
pd.set_option('max_columns',50) 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Here We are creating the module that will take care of our csv file. One class is designed to deal with csv files, the other class to deal with a postgres database and the last one to fetch the file online:

In [5]:
class Ingestion(object):
    """This is the ingestion class to deal with csv directly from the same directory where the module is"""

    def __init__(self, file, sep = ",", header = 0 ):
        self.file = file
        self.delimiter = sep
        self.df = pd.read_csv(file, sep= sep, header=header, engine='python', 
                              parse_dates = False)

    def file_csv(self):
        return self.df

class IngestionDatabase(object):

    """ This is the ingestion class to deal with postgress database """
    def __init__(self, database, query):
        self.engine = create_engine(database)
        self.table_names = self.engine.table_names()
        self.con = self.engine.connect()
        self.rs = self.con.execute(query)
        self.df = pd.DataFrame(self.rs.fetchmany(size=15))

    def cols(self):
        self.df.columns = self.rs.keys()
        return self.df

class OnlineFetch(object):
    
    def __init__(self, URL, fname="dc-crimes-search-results.csv"):
        self.URL = URL
        self.fname = fname 
        
  
    def fetch_data(self):
        
        """
        Helper method to retrieve the ML Repository dataset.
        """
        self.response = requests.get(self.URL)
        self.outpath  = os.path.abspath(self.fname)
        
        with open(self.outpath, 'wb') as f:
            f.write(self.response.content)
        self.online_data = pd.read_csv(self.outpath, engine='python', parse_dates = False)
        return self.online_data
        

In [8]:
# Fetching online data:

#URL = "https://datagate.dc.gov/search/open/crimes?daterange=8years&details=true&format=csv"
#obj = OnlineFetch(URL)
#DATA = obj.fetch_data()

In [9]:
#type(DATA)

## Creating the Ingestion instances for the crime dataset and weather dataset:

In [8]:
ingest = Ingestion('DC_Crime_Official2.csv')
data = ingest.file_csv()

In [9]:
ingest = Ingestion('dc_weather_2010-2018.csv')
weather_data = ingest.file_csv()

In [10]:
data.head(5)

,NEIGHBORHOOD_CLUSTER,CENSUS_TRACT,offensegroup,LONGITUDE,END_DATE,offense-text,SHIFT,YBLOCK,DISTRICT,WARD,YEAR,offensekey,BID,sector,PSA,ucr-rank,BLOCK_GROUP,VOTING_PRECINCT,XBLOCK,BLOCK,START_DATE,CCN,OFFENSE,ANC,REPORT_DAT,METHOD,location,LATITUDE
0,cluster 15,600.0,property,-77.069766,2015-05-05T17:45:00.000,theft f/auto,evening,140519.000000,2.0,3.0,2015,property|theft f/auto,NaN,2D2,204.0,7,000600 3,precinct 27,393951.000000,3500 - 3599 block of lowell street nw,2015-05-05T15:00:00.000,15064669,theft f/auto,3C,2015-05-05T22:06:00.000Z,others,"38.932540365033717,-77.069768169731375",38.932533
1,cluster 2,3000.0,violent,-77.031208,2015-05-05T20:55:00.000,robbery,evening,140084.000000,3.0,1.0,2015,violent|robbery,NaN,3D1,302.0,4,003000 2,precinct 39,397294.000000,1300 - 1399 block of irving street nw,2015-05-05T20:50:00.000,15064781,robbery,1A,2015-05-06T01:05:00.000Z,gun,"38.928638407177509,-77.031210110921407",38.928631
2,cluster 3,4400.0,property,-77.028460,2015-05-05T22:14:00.000,motor vehicle theft,evening,139201.000000,3.0,1.0,2015,property|motor vehicle theft,NaN,3D2,305.0,8,004400 1,precinct 22,397532.000000,1200 - 1247 block of florida avenue nw,2015-05-05T18:18:00.000,15064796,motor vehicle theft,1B,2015-05-06T02:58:00.000Z,others,"38.920684762113531,-77.028462123394547",38.920677
3,cluster 23,8804.0,violent,-76.985496,2015-06-23T08:00:00.000,homicide,midnight,137689.000000,5.0,5.0,2015,violent|homicide,NaN,5D3,506.0,1,008804 1,precinct 78,401258.000000,1200 - 1299 block of holbrook terrace ne,2015-06-23T05:23:00.000,15094190,homicide,5D,2015-06-24T04:00:00.000Z,gun,"38.907066722563066,-76.985498377563218",38.907059
4,cluster 25,8100.0,property,-76.990245,2015-06-23T06:30:00.000,theft f/auto,day,136398.650012,1.0,6.0,2015,property|theft f/auto,NaN,1D2,107.0,7,008100 1,precinct 81,400846.210015,duncan place ne and 12th street ne,2015-06-22T22:00:00.000,15094194,theft f/auto,6A,2015-06-23T12:48:00.000Z,others,"38.895443280295112,-76.990247632314322",38.895435


In [11]:
weather_data.head(5)

,Year,Month,Day,Hour,Temperature,Precipitation,Snowfall
0,2010,1,1,0,44.92,0.0,0.0
1,2010,1,1,1,43.83,0.0,0.0
2,2010,1,1,2,42.22,0.0,0.0
3,2010,1,1,3,40.23,0.0,0.0
4,2010,1,1,4,39.38,0.0,0.0


# -------------------------DATA WRANGLING--------------------

## 1. Wrangling crime data:

In [26]:
class Wrangling(object):

    def __init__(self, data = data):
        self.df = data
# drop empty rows
    def dropNA(self):
        self.df = self.df.dropna(how='all') # this only drop rows with 100% NA
        return self.df

    def __offense_column(self, text1 ='theft/other', text2 ='theft f/auto', text3 = 'assault w/dangerous weapon',
                       repl1 = 'theft', repl2 = 'auto theft', repl3 = 'assault with weapon' ):

        """There are 9 categories of offenses here:
        This function will transform the caterogies into more readable text
        for example : assault w/dangerous weapon = assault with dangerous weapon"""

        self.df['offense_text'] = self.df['offense_text'].replace([text1, text2, # add the column name to the arguments.
        text3], [repl1, repl2, repl3])
        return self.df

    def date_time_transformer(self, time = 'START_DATE', second_date = 'REPORT_DAT', third_date = 'END_DATE'):
        ''' transform into datetime 64 object and eliminate the second date column'''
        self.df[second_date] = pd.to_datetime(self.df['REPORT_DAT'])
        self.df[time] = pd.to_datetime(self.df['START_DATE'])
        self.df.drop([third_date], axis = 1, inplace = True)
        return self.df

    def __latlong_cutter(self):
        """ Reduce the presition of the lat long data by cutting them."""
        self.newlat = []
        self.newlon = []
        for item in self.df['LATITUDE']:
            item = str(item)
            item = float(item[0:6])
            self.newlat.append(item)

        self.df['LATITUDE'] = self.newlat

        for item in self.df['LONGITUDE']:
            item = str(item)
            item = float(item[0:7])
            self.newlon.append(item)
        self.df['LONGITUDE'] = self.newlon

        return self.df

    def lat_long_rounder(self, decimals = 3):
        """ Reduce the presition of the lat long data by rounging decimals"""
        self.df['LATITUDE'] = self.df['LATITUDE'].round(decimals = decimals)
        self.df['LONGITUDE'] = self.df['LONGITUDE'].round(decimals = decimals)
        return self.df

    def adress_format_modifier(self):
        """This columns replace some of the content from the block columns to it is easy to parse it"""

        self.splitted = []

        # creating the splited column
        # this is working. it cannot be transformed into pandas' .replace because it is using the split method
        # Note that the built in .replace it does not work properly with integers and neither with large amounts of
        # things to change.. This works but it is not very wise to use.
        for row in self.df['BLOCK']:
            row = row.replace("block of ", "")
            row = row.replace("street", "St")
            row = row.replace("-", "")
            row = row.split(' ', 1)
            self.splitted.append(row)
        self.df['splitted'] = self.splitted
        return self.df

    def block_parser(self):
        """ This is the block parser that separate block in start and en blocks"""

        self.startblock = []
        self.endblock_1 = []
        self.endblock = []
        #  create column 'startblock'
        for row in self.df['splitted']:
            row = row[0]
            self.startblock.append(row)
        self.df['startblock'] = self.startblock
        # create column  'endblock_1'
        for row in self.df['splitted']:
            row = row[-1].lstrip() # enblock_1
            row = row.split(' ',1)
            self.endblock_1.append(row)
        self.df['endblock_1'] = self.endblock_1
        # create column  'endblock'
        for row in self.df['endblock_1']:
            row = row[0]
            self.endblock.append(row)
        self.df['endblock'] = self.endblock
        return self.df

    def street_parser(self):
        self.street = []
        #creating column 'street'
        for row in self.df['endblock_1']:
            row = row[1]
            self.street.append(row)
        self.df['street'] = self.street
        return self.df

In [27]:
Wrangled = Wrangling()
Wrangled.dropNA()
Wrangled.date_time_transformer()
Wrangled.lat_long_rounder()
Wrangled.adress_format_modifier()
Wrangled.block_parser()
crime_df = Wrangled.street_parser()

In [28]:
crime_df["REPORT_DAT"] = pd.to_datetime(crime_df["REPORT_DAT"])

### 1.1: Set the REPORT_DAT column from the crime dataset as an index:

In [29]:
crime_df = crime_df.rename(columns = {'REPORT_DAT': 'datetime'})

In [30]:
crime_df["year"] =crime_df["START_DATE"].dt.year
crime_df["month"] =crime_df["START_DATE"].dt.month
crime_df["day"] =crime_df["START_DATE"].dt.day
crime_df["hour"] =crime_df["START_DATE"].dt.hour
crime_df["minute"] =crime_df["START_DATE"].dt.minute
crime_df["second"] =crime_df["START_DATE"].dt.second

#### 1.1.1 The maximum precision of the weather dataset is hourly, so we need to change the format:

In [31]:
crime_df['datetime'] = crime_df['datetime'].apply(lambda x: x.replace(minute = 0, second = 0))

#### 1.1.2: Adding index to crime dataset:

In [32]:
crime_df = crime_df.set_index(pd.DatetimeIndex(crime_df['datetime']), drop = False)

## 2. Wrangling weather data:

In [33]:
weather_data['datetime'] =  pd.to_datetime(weather_data[['Year', 'Month', 'Day', 'Hour']])

### 2.1 Seting index for weather data:

In [34]:
weather_data = weather_data.set_index(pd.DatetimeIndex(weather_data['datetime']), drop = False)

In [35]:
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 74064 entries, 2010-01-01 00:00:00 to 2018-06-13 23:00:00
Data columns (total 8 columns):
Year             74064 non-null int64
Month            74064 non-null int64
Day              74064 non-null int64
Hour             74064 non-null int64
Temperature      74064 non-null float64
Precipitation    74064 non-null float64
Snowfall         74064 non-null float64
datetime         74064 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(3), int64(4)
memory usage: 5.1 MB


## 3. Merging the two datasets:


First, we need to be sure that there are no duplicated values:

In [36]:
a= weather_data.index.duplicated(keep = False)
a = DataFrame(a)
b = a[a[0]== True]
b.count()

0    0
dtype: int64

### 3.1 Setting up the time range:

In [37]:
print(crime_df['datetime'].min())
print(crime_df['datetime'].max())

2010-06-11 04:00:00
2018-06-11 03:00:00


Sorting Values:

In [38]:
crime_df = crime_df.sort_values('datetime')
weather_data = weather_data.sort_values('datetime')

#### 3.1.1 Setting up the time range so it matches the crime data . The weather dataset contains dates that do not exist on the crime dataset. Also, the minimum common time unit is hour:

In [39]:
idx = pd.date_range('2010-06-11 04:00:00', '2018-06-11 03:00:00', freq = "H")

In [40]:
crime_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 282475 entries, 2010-06-11 04:00:00 to 2018-06-11 03:00:00
Data columns (total 38 columns):
NEIGHBORHOOD_CLUSTER    278891 non-null object
CENSUS_TRACT            281632 non-null float64
offensegroup            282475 non-null object
LONGITUDE               282475 non-null float64
offense-text            282475 non-null object
SHIFT                   282475 non-null object
YBLOCK                  282475 non-null float64
DISTRICT                282313 non-null float64
WARD                    282463 non-null float64
YEAR                    282475 non-null int64
offensekey              282475 non-null object
BID                     46124 non-null object
sector                  282296 non-null object
PSA                     282296 non-null float64
ucr-rank                282475 non-null int64
BLOCK_GROUP             281632 non-null object
VOTING_PRECINCT         282414 non-null object
XBLOCK                  282475 non-null float64
BLOCK

## 4. Merging both datasets:

In [41]:
df = pd.merge(crime_df, weather_data, how='left', on='datetime', left_index= True)

In [42]:
print(df.columns)

Index(['NEIGHBORHOOD_CLUSTER', 'CENSUS_TRACT', 'offensegroup', 'LONGITUDE',
       'offense-text', 'SHIFT', 'YBLOCK', 'DISTRICT', 'WARD', 'YEAR',
       'offensekey', 'BID', 'sector', 'PSA', 'ucr-rank', 'BLOCK_GROUP',
       'VOTING_PRECINCT', 'XBLOCK', 'BLOCK', 'START_DATE', 'CCN', 'OFFENSE',
       'ANC', 'datetime', 'METHOD', 'location', 'LATITUDE', 'splitted',
       'startblock', 'endblock_1', 'endblock', 'street', 'year', 'month',
       'day', 'hour', 'minute', 'second', 'Year', 'Month', 'Day', 'Hour',
       'Temperature', 'Precipitation', 'Snowfall'],
      dtype='object')


### 4.1 Saving all to disk:

In [43]:
with open('merged_dataset.pickle', 'wb') as dataframe:
    
    pickle.dump(df , dataframe)